## 수정사항
#### 2021.06.06
* T_ej는 마지막 조인트 각도에 영향을 받음, 하지만 Q는 unkown이므로 특정하는게 불가능, T_ee로 전부 새로 학습할 것.

In [1]:
from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession

config = ConfigProto()
config.gpu_options.allow_growth = True
session = InteractiveSession(config=config)

In [2]:
import os
import shutil
import random
PROJ_DIR = os.environ["RNB_PLANNING_DIR"]
os.chdir(os.path.join(PROJ_DIR, "src"))

from pkg.utils.utils_python3 import *
from pkg.planning.filtering.lattice_model.data_utils import *

DATA_PATH = os.path.join(PROJ_DIR, "data")
LAT_DATA_PATH = os.path.join(DATA_PATH, "latticized")
MODEL_PATH = os.path.join(PROJ_DIR, "model")
LAT_MODEL_PATH = os.path.join(MODEL_PATH,"latticized")
try_mkdir(MODEL_PATH)
try_mkdir(LAT_MODEL_PATH)
GRASP_FOLDER = "grasp"
ARM10_FOLDER = "arm_10"
ARM05_FOLDER = "arm_05"
FULLS_FOLDER = "full_scene"

In [3]:
ROBOT_TYPE_NAME="panda"
ROBOT_DATA_ROOT = os.path.join(LAT_DATA_PATH, ROBOT_TYPE_NAME)
ROBOT_DATA_ROOT_FAILMORE = ROBOT_DATA_ROOT+"-failmore"
ROBOT_MODEL_ROOT =  os.path.join(LAT_MODEL_PATH, ROBOT_TYPE_NAME)
ARM_FOLDER = ARM10_FOLDER

In [4]:
dataset_list = sorted(os.listdir(ROBOT_DATA_ROOT))
DATASET_TRAIN_FILTERED = dataset_list[:10]
DATASET_TEST_FILTERED = dataset_list[10:15]
print(DATASET_TRAIN_FILTERED)
print(DATASET_TEST_FILTERED)

dataset_list_fails = sorted([folder for folder 
                             in os.listdir(ROBOT_DATA_ROOT_FAILMORE) 
                             if not folder.startswith(".")])
DATASET_TRAIN_FAILMORE = dataset_list_fails[:10]
DATASET_TEST_FAILMORE = dataset_list_fails[10:15]
print(DATASET_TRAIN_FAILMORE)
print(DATASET_TEST_FAILMORE)

['20210219-091338', '20210219-124428', '20210219-234147', '20210220-035639', '20210220-080119', '20210220-122304', '20210220-160737', '20210220-194129', '20210220-234400', '20210221-043209']
['20210221-082144', '20210221-123619', '20210221-160542', '20210221-195509', '20210221-234239']
['20210606-091758', '20210606-100015', '20210606-103210', '20210606-111045', '20210606-114441', '20210606-121237', '20210606-125108', '20210606-132951', '20210606-141253', '20210606-144346']
['20210606-152059', '20210606-155651', '20210606-163145', '20210606-170529', '20210606-174213']


In [5]:
data_pairs_train_filtered = []
for dataset in DATASET_TRAIN_FILTERED:
    file_list = sorted(os.listdir(os.path.join(ROBOT_DATA_ROOT, dataset, GRASP_FOLDER)))
    for file in file_list:
        data_pairs_train_filtered.append((os.path.join(ROBOT_DATA_ROOT, dataset, GRASP_FOLDER, file), 
                                 os.path.join(ROBOT_DATA_ROOT, dataset, ARM_FOLDER, file)))
print("train set: {}".format(len(data_pairs_train_filtered)))        
        
        
data_pairs_test_filtered = []
for dataset in DATASET_TEST_FILTERED:
    file_list = sorted(os.listdir(os.path.join(ROBOT_DATA_ROOT, dataset, GRASP_FOLDER)))
    for file in file_list:
        data_pairs_test_filtered.append((os.path.join(ROBOT_DATA_ROOT, dataset, GRASP_FOLDER, file), 
                                 os.path.join(ROBOT_DATA_ROOT, dataset, ARM_FOLDER, file)))
print("test set: {}".format(len(data_pairs_test_filtered)))        

data_pairs_train_failmore = []
for dataset in DATASET_TRAIN_FAILMORE:
    file_list = sorted(os.listdir(os.path.join(ROBOT_DATA_ROOT_FAILMORE, dataset, GRASP_FOLDER)))
    for file in file_list:
        data_pairs_train_failmore.append((os.path.join(ROBOT_DATA_ROOT_FAILMORE, dataset, GRASP_FOLDER, file), 
                                 os.path.join(ROBOT_DATA_ROOT_FAILMORE, dataset, ARM_FOLDER, file)))
print("train fail set: {}".format(len(data_pairs_train_failmore)))        
        
        
data_pairs_test_failmore = []
for dataset in DATASET_TEST_FAILMORE:
    file_list = sorted(os.listdir(os.path.join(ROBOT_DATA_ROOT_FAILMORE, dataset, GRASP_FOLDER)))
    for file in file_list:
        data_pairs_test_failmore.append((os.path.join(ROBOT_DATA_ROOT_FAILMORE, dataset, GRASP_FOLDER, file), 
                                 os.path.join(ROBOT_DATA_ROOT_FAILMORE, dataset, ARM_FOLDER, file)))
print("test fail set: {}".format(len(data_pairs_test_failmore)))        

train set: 10047
test set: 5014
train fail set: 10026
test fail set: 5010


In [6]:
N_max_train = 10000
N_max_test = 5000

print("=== mix data ===")
data_pairs_train_mixed = mix_data_pairs(
    data_pairs_train_filtered, data_pairs_train_failmore, N_max_train)
data_pairs_test_mixed = mix_data_pairs(
    data_pairs_test_filtered, data_pairs_test_failmore, N_max_test)

data_pairs_train_filtered = data_pairs_train_filtered[:N_max_train]
data_pairs_test_filtered = data_pairs_test_filtered[:N_max_test]
data_pairs_train_failmore = data_pairs_train_failmore[:N_max_train]
data_pairs_test_failmore = data_pairs_test_failmore[:N_max_test]

print("="*20)
print("filtered data: {} / {}".format(
    len(data_pairs_train_filtered), 
    len(data_pairs_test_filtered)))
print("failmore data: {} / {}".format(
    len(data_pairs_train_failmore), 
    len(data_pairs_test_failmore)))
print("mixed data: {} / {}".format(
    len(data_pairs_train_mixed), 
    len(data_pairs_test_mixed)))

=== mix data ===
filtered success: 5000
success from failmore: 0
dataset fails: 5000
dataset all: 10000
filtered success: 2500
success from failmore: 0
dataset fails: 2500
dataset all: 5000
filtered data: 10000 / 5000
failmore data: 10000 / 5000
mixed data: 10000 / 5000


## Train

In [ ]:
from pkg.planning.filtering.lattice_model.lattice_model import *

# Create an instance of the model
model = ResNetModelTP()

loss_object = tf.keras.losses.BinaryCrossentropy()
optimizer = tf.keras.optimizers.Adam()

train_loss = tf.keras.metrics.Mean(name='train_loss')
train_accuracy = tf.keras.metrics.BinaryAccuracy(name='train_accuracy')
@tf.function
def train_step(images, labels):
    with tf.GradientTape() as tape:
        # training=True is only needed if there are layers with different
        # behavior during training versus inference (e.g. Dropout).
        predictions = model(images, training=True)
        loss = loss_object(labels, predictions)
    gradients = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))

    train_loss(loss)
    train_accuracy(labels, predictions)
    

test_loss = tf.keras.metrics.Mean(name='test_loss')
test_accuracy = tf.keras.metrics.BinaryAccuracy(name='test_accuracy')
@tf.function
def test_step(images, labels):
    # training=False is only needed if there are layers with different
    # behavior during training versus inference (e.g. Dropout).
    predictions = model(images, training=False)
    t_loss = loss_object(labels, predictions)

    test_loss(t_loss)
    test_accuracy(labels, predictions)

    
def train_script(data_pairs_train, data_pairs_test, robot_model_root,
                 EPOCHS_S=0, EPOCHS_E=15, BATCH_SIZE=16, LOG_STEP=100):
    current_time = get_now()
    logpath = os.path.join(robot_model_root, current_time)
    try_mkdir(logpath)
    train_log_dir = os.path.join(logpath, 'train')
    test_log_dir = os.path.join(logpath, 'test')
    model_log_dir = os.path.join(logpath, 'model_{}/')
    train_summary_writer = tf.summary.create_file_writer(train_log_dir)
    test_summary_writer = tf.summary.create_file_writer(test_log_dir)
    shutil.copy(os.path.join(
        PROJ_DIR,'src', 'pkg','planning','filtering','lattice_model','lattice_model.py' ), logpath)
    print(f'Log path: {logpath}')
    
    N_train = len(data_pairs_train)
    N_test = len(data_pairs_test)
    random.shuffle(data_pairs_test)
    res_list = []
    
    for epoch in range(EPOCHS_S, EPOCHS_E):
        # Reset the metrics at the start of the next epoch
        train_loss.reset_states()
        train_accuracy.reset_states()
        test_loss.reset_states()
        test_accuracy.reset_states()

        random.shuffle(data_pairs_train)
        i_step = 0
        data_batch, label_batch = [], []
        for data_pair in data_pairs_train:
            i_step += 1
            grasp_img, arm_img, rh_mask, label = load_data(data_pair)
            data_batch.append([grasp_img, arm_img, rh_mask])
            label_batch.append(label)
            if i_step%BATCH_SIZE==0:
                grasp_img_batch = np.array([grasp_img for grasp_img, arm_img, rh_mask in data_batch])
                arm_img_batch = np.array([arm_img for grasp_img, arm_img, rh_mask in data_batch])
                rh_mask_batch = np.array([rh_mask for grasp_img, arm_img, rh_mask in data_batch])
                label_batch = np.array(label_batch, dtype=np.int)
                train_step([grasp_img_batch, arm_img_batch, rh_mask_batch], label_batch)
                data_batch, label_batch = [], []
            if i_step%LOG_STEP==0:
                print("train step - {}/{}        ".format(i_step, N_train), end = '\r')
        with train_summary_writer.as_default():
            tf.summary.scalar('loss', train_loss.result(), step=epoch)
            tf.summary.scalar('accuracy', train_accuracy.result(), step=epoch)

        i_step = 0
        data_batch, label_batch = [], []
        for data_pair in data_pairs_test:
            i_step += 1
            grasp_img, arm_img, rh_mask, label = load_data(data_pair)
            data_batch.append([grasp_img, arm_img, rh_mask])
            label_batch.append(label)
            if i_step%BATCH_SIZE==0:
                grasp_img_batch = np.array([grasp_img for grasp_img, arm_img, rh_mask in data_batch])
                arm_img_batch = np.array([arm_img for grasp_img, arm_img, rh_mask in data_batch])
                rh_mask_batch = np.array([rh_mask for grasp_img, arm_img, rh_mask in data_batch])
                label_batch = np.array(label_batch, dtype=np.int)
                test_step([grasp_img_batch, arm_img_batch, rh_mask_batch], label_batch)
                data_batch, label_batch = [], []
            if i_step%LOG_STEP==0:
                print("test step - {}/{}        ".format(i_step, N_test), end = '\r')
        with test_summary_writer.as_default():
            tf.summary.scalar('loss', test_loss.result(), step=epoch)
            tf.summary.scalar('accuracy', test_accuracy.result(), step=epoch)

        model.save(model_log_dir.format(epoch + 1))

        print("")
        print("=================================================================")
        print(
            f'Epoch {epoch + 1}, '
            f'Loss: {train_loss.result()}, '
            f'Accuracy: {train_accuracy.result() * 100}, '
            f'Test Loss: {test_loss.result()}, '
            f'Test Accuracy: {test_accuracy.result() * 100}'
        )
        print("=================================================================")
        print("")
        res_list.append((float(test_accuracy.result())*100, float(test_loss.result())))
    return res_list

## train filtered data

In [ ]:
ROBOT_MODEL_ROOT_DIV = "{}-filtered".format(ROBOT_MODEL_ROOT)
data_pairs_train_div, data_pairs_test_div = \
        data_pairs_train_filtered, data_pairs_test_filtered

res_list = train_script(data_pairs_train_div, data_pairs_test_div, ROBOT_MODEL_ROOT_DIV)
print("{} acc/loss list:".format(ROBOT_MODEL_ROOT_DIV))
for i_r, res in enumerate(res_list):
    print("{}: {} %, {} ".format(i_r+1, np.round(res[0],2), np.round(res[1], 3)))

## failmore data

In [ ]:
ROBOT_MODEL_ROOT_DIV = "{}-failmore".format(ROBOT_MODEL_ROOT)
data_pairs_train_div, data_pairs_test_div = \
        data_pairs_train_failmore, data_pairs_test_failmore

res_list = train_script(data_pairs_train_div, data_pairs_test_div, ROBOT_MODEL_ROOT_DIV)
print("{} acc/loss list:".format(ROBOT_MODEL_ROOT_DIV))
for i_r, res in enumerate(res_list):
    print("{}: {} %, {} ".format(i_r+1, np.round(res[0],2), np.round(res[1], 3)))

## balanced data

In [ ]:
ROBOT_MODEL_ROOT_DIV = "{}-balanced".format(ROBOT_MODEL_ROOT)
data_pairs_train_div, data_pairs_test_div = \
        data_pairs_train_mixed, data_pairs_test_mixed

res_list = train_script(data_pairs_train_div, data_pairs_test_div, ROBOT_MODEL_ROOT_DIV)
print("{} acc/loss list:".format(ROBOT_MODEL_ROOT_DIV))
for i_r, res in enumerate(res_list):
    print("{}: {} %, {} ".format(i_r+1, np.round(res[0],2), np.round(res[1], 3)))

# Train results - Panda

#### filtered: 20210610-104017
```
1: 82.42 %, 0.417 
2: 81.87 %, 0.469 
3: 84.22 %, 0.36 
4: 85.46 %, 0.392 
5: 83.35 %, 0.39 
6: 84.11 %, 0.367 
7: 84.11 %, 0.38 
8: 83.77 %, 0.358 
9: 84.02 %, 0.356 
10: 84.26 %, 0.422 
11: 84.16 %, 0.378 
12: 85.03 %, 0.608 
13: 84.21 %, 0.396 
14: 84.11 %, 0.465 
15: 82.49 %, 0.717 
```

#### failmore: 20210610-110737
```
1: 96.14 %, 0.076 
2: 96.94 %, 0.068 
3: 96.07 %, 0.079 
4: 96.83 %, 0.068 
5: 96.79 %, 0.072 
6: 96.1 %, 0.08 
7: 96.27 %, 0.075 
8: 97.32 %, 0.066 
9: 96.77 %, 0.077 
10: 97.03 %, 0.075 
11: 96.88 %, 0.071 
12: 96.8 %, 0.081 
13: 97.35 %, 0.073 
14: 97.38 %, 0.078 
15: 97.62 %, 0.078 
```

#### balanced: 20210610-113433
```
1: 99.62 %, 0.011 
2: 99.88 %, 0.004 
3: 99.94 %, 0.002 
4: 99.98 %, 0.001 
5: 98.66 %, 0.043 
6: 99.9 %, 0.002 
7: 99.9 %, 0.003 
8: 99.98 %, 0.0 
9: 100.0 %, 0.0 
10: 100.0 %, 0.0 
11: 100.0 %, 0.0 
12: 100.0 %, 0.0 
13: 100.0 %, 0.0 
14: 99.09 %, 0.074 
15: 99.92 %, 0.002 
```

## New train 2021-06
* **[IMPORTANT] 이전에 Tej 사용 틀린 점 보완 했음** 

### Panda
* 20210606-080139 : first balanced train trial, 96.6% 달성 / 이전 데이터와 collision boundary shape 차이일 수 있음
* 20210606-082302 : 이전 데이터와 전부 섞어서 collision boundary shape 차이 영향 확인 - 87% 가량, 영향 확실, 재수집 필요
* 20210606-191455 : 새로 모았어도 분리 학습 하니 시작부터 99%, 문제 있음
* 20210606-194148 : 새로 모은거 분리 학습, 테스트만 순수 기존 데이터로 하니 59%, 문제 확실
* 20210606-201525 : 새 데이터만 학습, 96%? 하지만 fail bias 돼 있음.
* 20210606-204309 : 새 데이터만 학습, test set은 bias 제거함 ep8 89% 수준

### Indy7
* 20210607-224401 : 새 데이터만 학습, test set은 bias 제거함 ep12 79.5% 수준
* 20210610-052216 : 조인트 리밋 해제한 데이터만 학습, test set은 bias 제거, ep12 92% 수준

## Load & test

In [8]:
def get_tf_functions(model_root, model_folder, epoch_folder):
    # last_model = '20210606-204309' # panda
    # last_model = '20210610-052216' # indy
    # last_model = sorted(os.listdir(ROBOT_MODEL_ROOT))[-1]
    logpath = os.path.join(model_root, model_folder)

    # epochlist = sorted([item for item in os.listdir(logpath) if item.startswith("model")], key=lambda x: int(x[6:]))
    # print(epochlist)
    # last_save = epochlist[-1]
    model_log_dir = os.path.join(logpath, epoch_folder)

    import tensorflow as tf
    model = tf.keras.models.load_model(model_log_dir)

    @tf.function
    def inference(images):
        # training=False is only needed if there are layers with different
        # behavior during training versus inference (e.g. Dropout).
        predictions = model(images, training=False)
        return predictions

    loss_object = tf.keras.losses.BinaryCrossentropy()

    @tf.function
    def calc_loss(labels, predictions):
        # training=False is only needed if there are layers with different
        # behavior during training versus inference (e.g. Dropout).
        return loss_object(labels, predictions)
    return model, inference, calc_loss

def test_script(data_pairs_test, model_root, model_folder, epoch_folder):
    BATCH_SIZE = 1
    LOG_STEP = 100
    N_test = len(data_pairs_test)
    gtimer = GlobalTimer.instance()
    gtimer.reset()
    
    model, inference, calc_loss= get_tf_functions(model_root, model_folder, epoch_folder)

    grasp_img, arm_img, rh_mask, label = load_data(data_pairs_test[0])
    res = inference([np.array([grasp_img]), np.array([arm_img]), np.array([rh_mask])])


    i_step = 0
    res_list = []
    label_list = []
    loss_list= []
    grasp_img_batch = []
    arm_img_batch = []
    rh_mask_batch = []
    label_batch = [] 
    for data_pair in data_pairs_test:
        i_step += 1
        grasp_img, arm_img, rh_mask, label = load_data(data_pair)
        grasp_img_batch.append(grasp_img)
        arm_img_batch.append(arm_img)
        rh_mask_batch.append(rh_mask)
        label_batch.append(label)
        if len(grasp_img_batch)==BATCH_SIZE:
            grasp_img_batch, arm_img_batch, rh_mask_batch = np.array(grasp_img_batch), np.array(arm_img_batch), np.array(rh_mask_batch)
            with gtimer.block("inference"):
                res = inference([grasp_img_batch, arm_img_batch, rh_mask_batch])
            loss = calc_loss(label_batch, res)
            res_list = res_list + list(res.numpy()>0.5)
            label_list = label_list + label_batch
            loss_list.append(loss.numpy())
            grasp_img_batch = []
            arm_img_batch = []
            rh_mask_batch = []
            label_batch = [] 
        if i_step%LOG_STEP==0:
            print("test step - {}/{}        ".format(i_step, N_test), end = '\r')

    res_list = np.array(res_list)[:,1]
    label_list = np.array(label_list)[:,1]
    loss_list = np.array(loss_list)[:]

    acc = np.mean(np.equal(res_list, label_list)) * 100
    mean_loss = np.mean(loss_list)

    print("")
    print("=================================================================")
    print(
        f'Test Loss: {mean_loss} \n'
        f'Test Accuracy: {acc} \n'
        f'TP / FN / ACC: {np.sum(np.logical_and(res_list, label_list))}, ' 
        f'{np.sum(np.logical_and(np.logical_not(res_list), label_list))}, ' 
        f'{round(np.mean(res_list[np.where(label_list)])*100,2)}\n'
        f'FP / TN / ACC: {np.sum(np.logical_and(res_list, np.logical_not(label_list)))}, '
        f'{np.sum(np.logical_and(np.logical_not(res_list), np.logical_not(label_list)))}, '
        f'{round(np.mean(np.logical_not(res_list[np.where(np.logical_not(label_list))]))*100,2)}\n'
        f'PACC / NACC / TACC: {round(np.mean(label_list[np.where(res_list)])*100,2)}, '
        f'{round(np.mean(np.logical_not(label_list[np.where(np.logical_not(res_list))]))*100,2)}, '
        f'{round(np.mean(res_list==label_list)*100,2)}\n'
    )
    print("=================================================================")
    print("")
    print(gtimer)

    TP_FN_FP_TN = (np.sum(np.logical_and(res_list, label_list)), np.sum(np.logical_and(np.logical_not(res_list), label_list)), 
                   np.sum(np.logical_and(res_list, np.logical_not(label_list))), np.sum(np.logical_and(np.logical_not(res_list), np.logical_not(label_list))))
        
    return label_list, res_list

# Test all

In [9]:
print("========= Filtered =========")
model_foler, epoch_folder = '20210610-104017', 'model_4'
filtered_res = []
filtered_res.append(test_script(data_pairs_test_filtered, ROBOT_MODEL_ROOT+"-filtered", model_foler, epoch_folder))
filtered_res.append(test_script(data_pairs_test_failmore, ROBOT_MODEL_ROOT+"-filtered", model_foler, epoch_folder))
filtered_res.append(test_script(data_pairs_test_mixed, ROBOT_MODEL_ROOT+"-filtered", model_foler, epoch_folder))

print("========= Failmore =========")
model_foler, epoch_folder = '20210610-110737', 'model_15'
failmore_res = []
failmore_res.append(test_script(data_pairs_test_filtered, ROBOT_MODEL_ROOT+"-failmore", model_foler, epoch_folder))
failmore_res.append(test_script(data_pairs_test_failmore, ROBOT_MODEL_ROOT+"-failmore", model_foler, epoch_folder))
failmore_res.append(test_script(data_pairs_test_mixed, ROBOT_MODEL_ROOT+"-failmore", model_foler, epoch_folder))

print("========= Balanced =========")
model_foler, epoch_folder = '20210610-113433', 'model_15'
balanced_res = []
balanced_res.append(test_script(data_pairs_test_filtered, ROBOT_MODEL_ROOT+"-balanced", model_foler, epoch_folder))
balanced_res.append(test_script(data_pairs_test_failmore, ROBOT_MODEL_ROOT+"-balanced", model_foler, epoch_folder))
balanced_res.append(test_script(data_pairs_test_mixed, ROBOT_MODEL_ROOT+"-balanced", model_foler, epoch_folder))

print("========= Filtered =========")
print("filtered:")
print_confusion_mat(*filtered_res[0])
print("failmore:")
print_confusion_mat(*filtered_res[1])
print("balanced:")
print_confusion_mat(*filtered_res[2])
print("========================\n")

print("========= Failmore =========")
print("filtered:")
print_confusion_mat(*failmore_res[0])
print("failmore:")
print_confusion_mat(*failmore_res[1])
print("balanced:")
print_confusion_mat(*failmore_res[2])
print("========================\n")

print("========= Balanced =========")
print("filtered:")
print_confusion_mat(*balanced_res[0])
print("failmore:")
print_confusion_mat(*balanced_res[1])
print("balanced:")
print_confusion_mat(*balanced_res[2])
print("========================\n")

========= Filtered =========


test step - 5000/5000        
Test Loss: 0.3913080394268036 
Test Accuracy: 85.2 
TP / FN / ACC: 2673, 252, 91.38
FP / TN / ACC: 488, 1587, 76.48
PACC / NACC / TACC: 84.56, 86.3, 85.2


inference: 	127565.0 ms/5000 = 25.513 ms (10.381/41.004)



test step - 5000/5000        
Test Loss: 0.8329949378967285 
Test Accuracy: 68.36 
TP / FN / ACC: 172, 9, 95.03
FP / TN / ACC: 1573, 3246, 67.36
PACC / NACC / TACC: 9.86, 99.72, 68.36


inference: 	127350.0 ms/5000 = 25.47 ms (11.811/30.964)



test step - 5000/5000        
Test Loss: 0.5485051870346069 
Test Accuracy: 79.66 
TP / FN / ACC: 2273, 227, 90.92
FP / TN / ACC: 790, 1710, 68.4
PACC / NACC / TACC: 74.21, 88.28, 79.66


inference: 	127729.0 ms/5000 = 25.546 ms (10.221/31.68)

========= Failmore =========


test step - 5000/5000        
Test Loss: 1.454448938369751 
Test Accuracy: 66.46 
TP / FN / ACC: 1338, 1587, 45.74
FP / TN / ACC: 90, 1985, 95.66
PACC / NACC / TACC: 93.7, 55.57, 66.46


inference: 	128127.0 ms/5000 = 25.625 ms (11.684/31.42)



test step - 5000/5000        
Test Loss: 0.07828611880540848 
Test Accuracy: 97.58 
TP / FN / ACC: 104, 77, 57.46
FP / TN / ACC: 44, 4775, 99.09
PACC / NACC / TACC: 70.27, 98.41, 97.58


inference: 	128798.0 ms/5000 = 25.76 ms (12.115/31.827)



test step - 5000/5000        
Test Loss: 1.1915452480316162 
Test Accuracy: 72.64 
TP / FN / ACC: 1158, 1342, 46.32
FP / TN / ACC: 26, 2474, 98.96
PACC / NACC / TACC: 97.8, 64.83, 72.64


inference: 	128689.0 ms/5000 = 25.738 ms (12.217/33.968)

========= Balanced =========


test step - 5000/5000        
Test Loss: 4.108362197875977 
Test Accuracy: 58.74 
TP / FN / ACC: 2925, 0, 100.0
FP / TN / ACC: 2063, 12, 0.58
PACC / NACC / TACC: 58.64, 100.0, 58.74


inference: 	128773.0 ms/5000 = 25.755 ms (12.782/31.845)



test step - 5000/5000        
Test Loss: 0.30704060196876526 
Test Accuracy: 96.3 
TP / FN / ACC: 1, 180, 0.55
FP / TN / ACC: 5, 4814, 99.9
PACC / NACC / TACC: 16.67, 96.4, 96.3


inference: 	128515.0 ms/5000 = 25.703 ms (12.191/31.163)



test step - 5000/5000        
Test Loss: 0.001708336640149355 
Test Accuracy: 99.92 
TP / FN / ACC: 2500, 0, 100.0
FP / TN / ACC: 4, 2496, 99.84
PACC / NACC / TACC: 99.84, 100.0, 99.92


inference: 	128326.0 ms/5000 = 25.665 ms (11.319/31.504)

========= Filtered =========
filtered:
	 PP 	 	 PN 	 	 5000
GP 	 2673 	 	 252 	 	 91.38%
GN 	 488 	 	 1587 	 76.48%
AL 	 84.56% 	 86.30% 	 85.20%
failmore:
	 PP 	 	 PN 	 	 5000
GP 	 172 	 	 9 	 	 95.03%
GN 	 1573 	 	 3246 	 67.36%
AL 	 9.86% 	 99.72% 	 68.36%
balanced:
	 PP 	 	 PN 	 	 5000
GP 	 2273 	 	 227 	 	 90.92%
GN 	 790 	 	 1710 	 68.40%
AL 	 74.21% 	 88.28% 	 79.66%

========= Failmore =========
filtered:
	 PP 	 	 PN 	 	 5000
GP 	 1338 	 	 1587 	 	 45.74%
GN 	 90 	 	 1985 	 95.66%
AL 	 93.70% 	 55.57% 	 66.46%
failmore:
	 PP 	 	 PN 	 	 5000
GP 	 104 	 	 77 	 	 57.46%
GN 	 44 	 	 4775 	 99.09%
AL 	 70.27% 	 98.41% 	 97.58%
balanced:
	 PP 	 	 PN 	 	 5000
GP 	 1158 	 	 1342 	 	 46.32%
GN 	 26 	 	 2474 	 98.96%
AL 	 97.80% 	 64.83% 	 72.64%


### Test result - panda
```
inference: 	128326.0 ms/5000 = 25.665 ms (11.319/31.504)

========= Filtered =========
filtered:
	 PP 	 	 PN 	 	 5000
GP 	 2673 	 	 252 	 	 91.38%
GN 	 488 	 	 1587 	 76.48%
AL 	 84.56% 	 86.30% 	 85.20%
failmore:
	 PP 	 	 PN 	 	 5000
GP 	 172 	 	 9 	 	 95.03%
GN 	 1573 	 	 3246 	 67.36%
AL 	 9.86% 	 99.72% 	 68.36%
balanced:
	 PP 	 	 PN 	 	 5000
GP 	 2273 	 	 227 	 	 90.92%
GN 	 790 	 	 1710 	 68.40%
AL 	 74.21% 	 88.28% 	 79.66%
========================

========= Failmore =========
filtered:
	 PP 	 	 PN 	 	 5000
GP 	 1338 	 	 1587 	 	 45.74%
GN 	 90 	 	 1985 	 95.66%
AL 	 93.70% 	 55.57% 	 66.46%
failmore:
	 PP 	 	 PN 	 	 5000
GP 	 104 	 	 77 	 	 57.46%
GN 	 44 	 	 4775 	 99.09%
AL 	 70.27% 	 98.41% 	 97.58%
balanced:
	 PP 	 	 PN 	 	 5000
GP 	 1158 	 	 1342 	 	 46.32%
GN 	 26 	 	 2474 	 98.96%
AL 	 97.80% 	 64.83% 	 72.64%
========================

========= Balanced =========
filtered:
	 PP 	 	 PN 	 	 5000
GP 	 2925 	 	 0 	 	 100.00%
GN 	 2063 	 	 12 	 0.58%
AL 	 58.64% 	 100.00% 	 58.74%
failmore:
	 PP 	 	 PN 	 	 5000
GP 	 1 	 	 180 	 	 0.55%
GN 	 5 	 	 4814 	 99.90%
AL 	 16.67% 	 96.40% 	 96.30%
balanced:
	 PP 	 	 PN 	 	 5000
GP 	 2500 	 	 0 	 	 100.00%
GN 	 4 	 	 2496 	 99.84%
AL 	 99.84% 	 100.00% 	 99.92%
========================
```

## Test convergence

In [ ]:
last_model = sorted(os.listdir(ROBOT_MODEL_ROOT))[-1]
logpath = os.path.join(ROBOT_MODEL_ROOT, last_model)

model_epoch_list = []
acc_epoch_list = []
loss_epoch_list = []
# last_save = sorted([item for item in os.listdir(logpath) if item.startswith("model")])[-1]
# last_save = 'model_1'
for last_save in sorted([item for item in os.listdir(logpath) if item.startswith("model")], key=lambda x: int(x[6:])):
    model_log_dir = os.path.join(logpath, last_save)

    import tensorflow as tf
    model = tf.keras.models.load_model(model_log_dir)

    @tf.function
    def inference(images):
        # training=False is only needed if there are layers with different
        # behavior during training versus inference (e.g. Dropout).
        predictions = model(images, training=False)
        return predictions

    loss_object = tf.keras.losses.BinaryCrossentropy()

    @tf.function
    def calc_loss(labels, predictions):
        # training=False is only needed if there are layers with different
        # behavior during training versus inference (e.g. Dropout).
        return loss_object(labels, predictions)
    
    BATCH_SIZE = 50
    LOG_STEP = 100
    N_test = len(data_pairs_test)
    gtimer = GlobalTimer.instance()
    gtimer.reset()

    i_step = 0
    res_list = []
    label_list = []
    loss_list= []
    grasp_img_batch = []
    arm_img_batch = []
    rh_mask_batch = []
    label_batch = [] 
    for data_pair in data_pairs_test:
        i_step += 1
        grasp_img, arm_img, rh_mask, label = load_data(data_pair)
        grasp_img_batch.append(grasp_img)
        arm_img_batch.append(arm_img)
        rh_mask_batch.append(rh_mask)
        label_batch.append(label)
        if len(grasp_img_batch)==BATCH_SIZE:
            grasp_img_batch, arm_img_batch, rh_mask_batch = np.array(grasp_img_batch), np.array(arm_img_batch), np.array(rh_mask_batch)
            with gtimer.block("inference"):
                res = inference([grasp_img_batch, arm_img_batch, rh_mask_batch])
            loss = calc_loss(label_batch, res)
            res_list = res_list + list(res.numpy()>0.5)
            label_list = label_list + label_batch
            loss_list.append(loss.numpy())
            grasp_img_batch = []
            arm_img_batch = []
            rh_mask_batch = []
            label_batch = [] 
        if i_step%LOG_STEP==0:
            print("test step - {}/{}        ".format(i_step, N_test), end = '\r')

    res_list = np.array(res_list)[:5000,1]
    label_list = np.array(label_list)[:5000,1]
    loss_list = np.array(loss_list)[:5000]
    
    acc = np.mean(np.equal(res_list, label_list)) * 100
    mean_loss = np.mean(loss_list)

    print("")
    print("=================================================================")
    print(
        f'Test Loss: {mean_loss} \n'
        f'Test Accuracy: {acc} \n'
        f'TP / FN / ACC: {np.sum(np.logical_and(res_list, label_list))}, ' 
        f'{np.sum(np.logical_and(np.logical_not(res_list), label_list))}, ' 
        f'{round(np.mean(res_list[np.where(label_list)])*100,2)}\n'
        f'FP / TN / ACC: {np.sum(np.logical_and(res_list, np.logical_not(label_list)))}, '
        f'{np.sum(np.logical_and(np.logical_not(res_list), np.logical_not(label_list)))}, '
        f'{round(np.mean(np.logical_not(res_list[np.where(np.logical_not(label_list))]))*100,2)}\n'
        f'PACC / NACC / TACC: {round(np.mean(label_list[np.where(res_list)])*100,2)}, '
        f'{round(np.mean(np.logical_not(label_list[np.where(np.logical_not(res_list))]))*100,2)}, '
        f'{round(np.mean(res_list==label_list)*100,2)}\n'
    )
    print("=================================================================")
    print("")
    print(gtimer)
    model_epoch_list.append(last_save)
    acc_epoch_list.append(acc)
    loss_epoch_list.append(mean_loss)

In [ ]:
import matplotlib.pyplot as plt
print(model_epoch_list)
plt.figure(figsize=(10,3))
plt.subplot(1,2,1)
plt.plot(acc_epoch_list)
plt.subplot(1,2,2)
plt.plot(loss_epoch_list)

In [ ]:
save_json(ROBOT_TYPE_NAME+".json", {"epoch": np.array(model_epoch_list), "acc": np.array(acc_epoch_list), "loss": np.array(loss_epoch_list)})

## test on shared array

In [ ]:
import SharedArray as sa
BATCH_SIZE = 1
SERVER_PERIOD = 1e-2
# Create an array in shared memory.
robot_type_p = sa.attach("shm://robot_type")
grasp_img_p = sa.attach("shm://grasp_img")
arm_img_p = sa.attach("shm://arm_img")
rh_mask_p = sa.attach("shm://rh_mask")
result_p = sa.attach("shm://result")
query_in = sa.attach("shm://query_in")
response_out = sa.attach("shm://response_out")
query_quit = sa.attach("shm://query_quit")

def query_wait_response(grasp_img_batch, arm_img_batch, rh_mask_batch):
    grasp_img_p[:] = grasp_img_batch[:]
    arm_img_p[:] = arm_img_batch[:]
    rh_mask_p[:] = rh_mask_batch[:]
    query_in[0] = True
    while not response_out[0]:
        time.sleep(SERVER_PERIOD)
    response_out[0] = False
    return np.copy(result_p)

def quit_shared_server():
    query_quit[0] = True

In [ ]:
LOG_STEP = 100
N_test = len(data_pairs_test)

i_step = 0
data_batch, label_batch = [], []
result_list = []
label_list = []
for data_pair in data_pairs_test:
    i_step += 1
    grasp_img, arm_img, rh_mask, label = load_data(data_pair)
    data_batch.append([grasp_img, arm_img, rh_mask])
    label_batch.append(label)
    if i_step%BATCH_SIZE==0:
        grasp_img_batch = np.array([grasp_img for grasp_img, arm_img, rh_mask in data_batch])
        arm_img_batch = np.array([arm_img for grasp_img, arm_img, rh_mask in data_batch])
        rh_mask_batch = np.array([rh_mask for grasp_img, arm_img, rh_mask in data_batch])
        label_batch = np.array(label_batch, dtype=np.int)
        result = query_wait_response(grasp_img_batch, arm_img_batch, rh_mask_batch)
        result_list.append(result)
        label_list.append(label_batch)
        data_batch, label_batch = [], []
    if i_step%LOG_STEP==0:
        print("test step - {}/{}        ".format(i_step, N_test), end = '\r')

In [ ]:
result_all = np.array(result_list)
label_all = np.array(label_list)

In [ ]:
np.mean(np.round(result_all).astype(np.int) == label_all.astype(np.int), axis=0)

In [ ]:
quit_shared_server()

## save gridded data

In [ ]:
data_pair = data_pairs_test[0]
grasp_data = load_pickle(data_pair[0])
arm_data = load_pickle(data_pair[1])
grasp_tar_idx = grasp_data[b'tar']
grasp_tool_idx = grasp_data[b'tool']
arm_tar_idx = arm_data[b'tar']
Tee = grasp_data[b'T_end_effector']
Tej = grasp_data[b'T_end_joint']
Tref_base = grasp_data[b'Tref_base']
reach_lb = grasp_data[b'reach']
retrieve_lb = grasp_data[b'retrieve']
r, th, h = cart2cyl(*Tee[:3,3])
r_ej, th, h_ej = cart2cyl(*Tej[:3,3])
r_class = div_r(r_ej)
h_class = div_h(h_ej)
r_mask = np.zeros(RH_MASK_SIZE)
r_mask[r_class*RH_MASK_STEP:r_class*RH_MASK_STEP+RH_MASK_STEP] = 1
h_mask = np.zeros(RH_MASK_SIZE)
h_mask[h_class*RH_MASK_STEP:h_class*RH_MASK_STEP+RH_MASK_STEP] = 1
rh_mask = np.concatenate([r_mask, h_mask])
# r_ej_list.append(r_ej)
# h_ej_list.append(h_ej)
# reach_lb_list.append(reach_lb)
Tref = SE3(Rot_axis(3, th), Tee[:3,3])
grasp_tool_img = np.zeros(GRASP_SHAPE)
grasp_tar_img = np.zeros(GRASP_SHAPE)
grasp_tool_img[np.unravel_index(grasp_tool_idx, shape=GRASP_SHAPE)] = 1
grasp_tar_img[np.unravel_index(grasp_tar_idx, shape=GRASP_SHAPE)] = 1
arm_img = np.zeros(ARM_SHAPE+(1,))
arm_img[np.unravel_index(arm_tar_idx, shape=ARM_SHAPE)] = 1
grasp_img = np.stack([grasp_tool_img, grasp_tar_img], axis=-1)

In [ ]:
# save_json("tar_arm.json", np.array(np.unravel_index(arm_tar_idx, shape=ARM_SHAPE)).transpose())

In [ ]:
# save_json("tool_effector.json", np.array(np.unravel_index(grasp_tool_idx, shape=GRASP_SHAPE)).transpose())
# save_json("tar_effector.json", np.array(np.unravel_index(grasp_tar_idx, shape=GRASP_SHAPE)).transpose())

## visualize r, h class distribution

In [ ]:
import matplotlib.pyplot as plt
r_ej_list = []
h_ej_list = []
for data_pair in data_pairs_train:
    grasp_data = load_pickle(data_pair[0])
    arm_data = load_pickle(data_pair[1])
    grasp_obj_idx = grasp_data[b'obj']
    grasp_tar_idx = grasp_data[b'tar']
    grasp_tool_idx = grasp_data[b'tool']
    arm_tar_idx = arm_data[b'tar']
    Tee = grasp_data[b'T_end_effector']
    Tej = grasp_data[b'T_end_joint']
    Tref_base = grasp_data[b'Tref_base']
    reach_lb = grasp_data[b'reach']
    retrieve_lb = grasp_data[b'retrieve']
    r, th, h = cart2cyl(*Tee[:3,3])
    r_ej, th, h_ej = cart2cyl(*Tej[:3,3])
    r_ej_list.append(r_ej)
    h_ej_list.append(h_ej)

In [ ]:
def div_r(r):
    return floor(sigmoid((r)/0.1-7)*8)

def div_h(h):
    return floor(sigmoid((h+0.6)/0.2-5)*8)

In [ ]:
plt.subplot(1,2,1)
plt.plot(sorted([div_r(r_ej_) for r_ej_ in r_ej_list]), '.')
plt.subplot(1,2,2)
plt.plot(sorted([div_h(h_ej_) for h_ej_ in h_ej_list]), '.')

In [ ]:
plt.subplot(1,2,1)
plt.plot(sorted([div_r(r_ej_) for r_ej_ in r_ej_list]), '.')
plt.subplot(1,2,2)
plt.plot(sorted([div_h(h_ej_) for h_ej_ in h_ej_list]), '.')

In [ ]:

plt.plot(sorted(np.floor(np.divide(r_ej_list,0.1))), '.')
plt.plot(sorted(np.floor(np.divide(h_ej_list,0.1))), '.')

In [ ]:

plt.plot(sorted(r_ej_list), '.')
plt.plot(sorted(h_ej_list), '.')

In [ ]:
np.max(r_ej_list)

In [ ]:
1.0 - 0.2

In [ ]:
np.min((np.array(r_ej_list)-0.2))

In [ ]:
np.min(np.floor((np.array(r_ej_list)-0.2)/0.05).astype(np.int))

In [ ]:
def gaussian(x, mu, sig):
    return np.exp(-np.power(x - mu, 2.) / (2 * np.power(sig, 2.)))

In [ ]:
x = div_r_gaussian(sorted(r_ej_list)[5000])

In [ ]:
y = div_h_gaussian(sorted(h_ej_list)[5000])

In [ ]:
plt.figure(figsize=(11,5))
plt.imshow(x[np.newaxis, :], cmap='gray')

In [ ]:
plt.figure(figsize=(16,5))
plt.imshow(y[np.newaxis, :], cmap='gray')

In [ ]:
def div_h_gaussian(h_val):
    return gaussian(h_val, np.arange(-0.5,1.1, 0.05),0.05)

In [ ]:

def div_h(h):
    return floor(sigmoid((h+0.6)/0.2-4.5)*8)

In [ ]:
X = np.repeat(np.transpose([div_r_gaussian(r_val) for r_val in sorted(r_ej_list)]), 100, axis=0)

In [ ]:
X = np.repeat(np.transpose([div_h_gaussian(h_val) for h_val in sorted(h_ej_list)]), 100, axis=0)

In [ ]:
plt.figure(figsize=(15,15))
plt.imshow(X)

In [ ]:
min(r_ej_list), max(r_ej_list)

In [ ]:
min(h_ej_list), max(h_ej_list)

In [ ]:
plt.plot(sorted(np.clip(np.floor((np.array(r_ej_list)-0.2)/0.05).astype(np.int), 0, 15)))

In [ ]:
Ndat = len(data_pairs_train)
sorted(r_ej_list)